X是要分析的列，需要drop target列和不相关的列
y是目标列

In [ ]:
X = df.drop(['target','trayno'],axis=1)
y = df['target']

创建决策树模型，打印准确率。

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(f"准确率: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred))

classification_report 分类报告

准确率（Accuracy）:
模型在所有样本中预测正确的比例。

每个类别的指标

* precision（精确率）：预测为正类的样本中，实际为正类的比例。
* recall（召回率）：实际为正类的样本中，预测为正类的比例。
* f1-score（F1 分数）：精确率和召回率的调和平均值。
* support（支持数）：每个类别的样本数量。
样本数量接近，说明数据集较为平衡。

整体指标

* accuracy（准确率）：模型在所有样本中预测正确的比例。
* macro avg（宏平均）：所有类别的指标的平均值。
* weighted avg（加权平均）：根据每个类别的样本数量加权计算的指标平均值。

适用性：如果测试集与真实数据分布一致，模型在实际应用中可能表现良好。

# 特征重要性分析

In [ ]:
def feature_importance(model):
    importance = model.feature_importances_
    feature_names = X.columns

    # 创建特征重要性DataFrame
    feature_importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': importance
    })

    # 按重要性排序
    feature_importance_df = feature_importance_df.sort_values('Importance', ascending=False)
    print("参数重要性:")
    print(feature_importance_df)

In [ ]:
# 获取特征重要性
importance = model.feature_importances_
feature_names = X.columns

# 创建特征重要性DataFrame
feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importance
})

# 按重要性排序
feature_importance_df = feature_importance_df.sort_values('Importance', ascending=False)

# 可视化特征重要性
plt.figure(figsize=(10, 6))
plt.barh(feature_importance_df['Feature'], feature_importance_df['Importance'])
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.title('Feature Importance for Quality Prediction')
plt.tight_layout()
plt.show()

# 打印前几个最重要的特征
print("最重要的参数:")
print(feature_importance_df.head())

# 可视化决策树

In [ ]:
from sklearn import tree
# 可视化决策树
plt.figure(figsize=(20, 10))
tree.plot_tree(model, feature_names=feature_names, 
               class_names=['ac', 'nc'], 
               filled=True, rounded=True)
plt.show()

决策树分析

出现在较高层的特征通常更重要，因为它们对样本的分割影响更大。

节点结构，第一个节点为例子，解释信息：

* 特征分割条件：duration_type<=31.5，表示根据duration_type的值是否小于等于31.5进行分割。
* 基尼指数（Gini Index）：gini = 0.3，表示节点的不纯度。

    gini 值越小：表示节点越纯净，样本更倾向于属于同一类别。

    gini 值越大：表示节点越不纯，样本分布越均匀。
    
* 样本数量：samples=989，表示当前节点包含 989 个样本。
* 类别分布：value=[60, 40]，表示当前节点中类别 A 有 469 个样本，类别 B 有 520 个样本。
* 类别标签：class=nc，表示当前节点的预测类别。

颜色填充

* 颜色深浅：颜色越深，表示当前节点的纯度越高（即某一类别的样本占比越大）。
* 颜色对应类别：不同颜色代表不同类别。

# 优化模型

In [ ]:
import numpy as np

# 优化模型

# 剪枝：避免过拟合
# 使用剪枝参数构建更稳健的模型
optimized_model = DecisionTreeClassifier(max_depth=5, min_samples_split=10, min_samples_leaf=5, random_state=42)
optimized_model.fit(X_train, y_train)

# 交叉验证：提高模型稳定性
from sklearn.model_selection import cross_val_score

scores = cross_val_score(optimized_model, X, y, cv=5)
print(f"交叉验证平均准确率: {np.mean(scores)}")